# **Variant filtering example**
This Jupyter script is an example for a use case of the Oskar API, which works in perfect combination with Pyspark in order to provide a genomic analysis tool.

First of all we need to import the modules that contain the Oskar API as well as the Spark ones. Also we need to create a new instance of the Oskar object, from which depends the whole functionality, and load our data as "df".

In [4]:
from pyoskar.core import Oskar
from pyoskar.sql import *
from pyoskar.analysis import *
from pyspark.sql.functions import col, udf, count, explode, concat, when, expr
from pyspark.sql.functions import *

oskar = Oskar(spark)
df = oskar.load("/home/roldanx/appl/oskar/oskar-spark/src/test/resources/platinum_chr22.small.parquet")
df.createOrReplaceTempView("platinum")

**Optional: Create a Temporary View case you plan to acces the data folder from SQL.*

## Region filter
First of all we will execute a simple filtering based on a restricted zone for those people who are not familiarizes with spark sintax. In this example we chose the 22th chromosome and the nucleotides between 17.000.000 and 17.500.000 position. This would be two of the statements we could use:

In [2]:
# Pyspark
df.select("id", "chromosome", "start", "end").filter(df.chromosome == 22).filter(df.start > 17000000).filter(df.end < 17500000).show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



In [4]:
# SQL
spark.sql("SELECT id,chromosome,start,end FROM platinum WHERE chromosome =='22' AND start > 17000000 AND end < 17500000").show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



## Gene
We may want to execute a filtering which ties up the variants attached to a concrete gene. "NBEAP3" was chosen as the target. Here we start preciating the functionality of Oskar API: "genes" UDF will automatically locate and acces the field with that same name. We just need to specify the column where we store the gene info: "annotation". Now it's only left to define the filter.

In [6]:
df.select("id", genes("annotation").alias("genes")).filter(array_contains("genes", "NBEAP3")).show()

+---------------+--------------------+
|             id|               genes|
+---------------+--------------------+
|22:16096040:G:A|            [NBEAP3]|
|22:16099957:C:T|            [NBEAP3]|
|22:16100462:A:G|            [NBEAP3]|
|22:16105660:G:A|            [NBEAP3]|
|22:16112391:G:A|            [NBEAP3]|
|22:16114913:A:T|            [NBEAP3]|
|22:16127471:A:-|[LA16c-60H5.7, NB...|
+---------------+--------------------+



## Genotype
In case we want to get the genotypes corresponding to any sample we could use the "genotype" function among others like "samples_data_field". We will also need to specify the field where that info is contained ("studies") just as "genes" function, the sample and the format.

In [7]:
# Spark
df.select("id", genotype("studies", "NA12877").alias("NA12877")).show(10)

+---------------+-------+
|             id|NA12877|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    0/1|
|22:16077310:T:A|    ./.|
|22:16080499:A:G|    0/1|
|22:16084621:T:C|    ./.|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    0/1|
+---------------+-------+
only showing top 10 rows



In [5]:
# SQL
spark.sql("SELECT id, genotype(studies, 'NA12877')  AS NA12877 FROM platinum").show(10)

+---------------+-------+
|             id|NA12877|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    0/1|
|22:16077310:T:A|    ./.|
|22:16080499:A:G|    0/1|
|22:16084621:T:C|    ./.|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    0/1|
+---------------+-------+
only showing top 10 rows



## Population Frequency
In order to get the population frequency data we need to use the "population_frequency" UDF. Apart from specifying the field were we contain that information, we will need to specify the ID of the concrete study we would like to check and the target population.

In [28]:
df.select("id", population_frequency("annotation", "GNOMAD_GENOMES", "ALL").alias("GNOMAD_GENOMES:ALL"))\
    .filter(population_frequency("annotation", "GNOMAD_GENOMES", "ALL") != 0).show(10)

+---------------+--------------------+
|             id|  GNOMAD_GENOMES:ALL|
+---------------+--------------------+
|22:16054454:C:T| 0.07566695660352707|
|22:16065809:T:C| 0.14594951272010803|
|22:16077310:T:A|  0.2338419109582901|
|22:16091610:G:T|0.003129890421405...|
|22:16099957:C:T|  0.6782668232917786|
|22:16105660:G:A| 0.12387744337320328|
|22:16114913:A:T| 0.11458797007799149|
|22:16127471:A:-|0.004762233234941959|
|22:16134019:G:T| 0.03758351877331734|
|22:16138943:C:G|  0.2861359417438507|
+---------------+--------------------+
only showing top 10 rows



Otherwise, if we are interested in getting all the Population Frequencies available without caring the population, we can use this other method named "population_frequency_as_map" which will return the whole set as a dictionary format. Then we could apply "explode" and convert that dictionary into a new dataframe. In this example we have also filtered by variant to get a better-looking output:

In [29]:
PF = df.select(df.id, population_frequency_as_map("annotation").alias("populationFrequencies"))\
    .filter(df.id == "22:16099957:C:T")
PF.select("id", explode(PF.populationFrequencies).alias("study", "populationFrequenciesDF")).show()

+---------------+--------------------+-----------------------+
|             id|               study|populationFrequenciesDF|
+---------------+--------------------+-----------------------+
|22:16099957:C:T|  GNOMAD_GENOMES:OTH|     0.6896551847457886|
|22:16099957:C:T|  GNOMAD_GENOMES:ALL|     0.6782668232917786|
|22:16099957:C:T|  GNOMAD_GENOMES:AFR|     0.6747621297836304|
|22:16099957:C:T|  GNOMAD_GENOMES:NFE|     0.6699579954147339|
|22:16099957:C:T| GNOMAD_GENOMES:MALE|      0.682662844657898|
|22:16099957:C:T|  GNOMAD_GENOMES:FIN|     0.6726332306861877|
|22:16099957:C:T|  GNOMAD_GENOMES:EAS|     0.9523809552192688|
|22:16099957:C:T|  GNOMAD_GENOMES:ASJ|     0.5721649527549744|
|22:16099957:C:T|GNOMAD_GENOMES:FE...|     0.6727412939071655|
|22:16099957:C:T|  GNOMAD_GENOMES:AMR|     0.6950549483299255|
+---------------+--------------------+-----------------------+



## Consequence type
Theres two ways we could acces consequence type field. The first one is the "consequence_types" UDF.

In [27]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types("annotation").alias("CT")).show(10)

+---------------+--------------------+--------------------+
|             id|               genes|                  CT|
+---------------+--------------------+--------------------+
|22:16054454:C:T|                  []|[intergenic_variant]|
|22:16065809:T:C|       [LA16c-4G1.3]|[regulatory_regio...|
|22:16077310:T:A|       [LA16c-4G1.4]|[2KB_upstream_var...|
|22:16080499:A:G|[LA16c-4G1.4, LA1...|[upstream_gene_va...|
|22:16084621:T:C|       [LA16c-4G1.5]|[TF_binding_site_...|
|22:16091610:G:T|                  []|[intergenic_variant]|
|22:16096040:G:A|            [NBEAP3]|[downstream_gene_...|
|22:16099957:C:T|            [NBEAP3]|[2KB_downstream_v...|
|22:16100462:A:G|            [NBEAP3]|[2KB_downstream_v...|
|22:16105660:G:A|            [NBEAP3]|[non_coding_trans...|
+---------------+--------------------+--------------------+
only showing top 10 rows



The "consequence_types_by_gene" UDF, unlike "consequence_types", will return the field that contains the consequence types that directly match the specified gene.

In [17]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types_by_gene("annotation", "NBEAP3").alias("CT")).show(10, False)

+---------------+--------------------+--------------------+
|             id|               genes|                  CT|
+---------------+--------------------+--------------------+
|22:16054454:C:T|                  []|                  []|
|22:16065809:T:C|       [LA16c-4G1.3]|                  []|
|22:16077310:T:A|       [LA16c-4G1.4]|                  []|
|22:16080499:A:G|[LA16c-4G1.4, LA1...|                  []|
|22:16084621:T:C|       [LA16c-4G1.5]|                  []|
|22:16091610:G:T|                  []|                  []|
|22:16096040:G:A|            [NBEAP3]|[downstream_gene_...|
|22:16099957:C:T|            [NBEAP3]|[2KB_downstream_v...|
|22:16100462:A:G|            [NBEAP3]|[2KB_downstream_v...|
|22:16105660:G:A|            [NBEAP3]|[non_coding_trans...|
+---------------+--------------------+--------------------+
only showing top 10 rows



Then we may want to combine that output with a filter like this:

In [24]:
df.select(df.id, consequence_types_by_gene("annotation", "NBEAP3").alias("CT")).filter(array_contains("CT", "intron_variant")).show(10, False)

+---------------+-----------------------------------------------+
|id             |CT                                             |
+---------------+-----------------------------------------------+
|22:16105660:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16112391:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16114913:A:T|[non_coding_transcript_variant, intron_variant]|
+---------------+-----------------------------------------------+



## Protein substitution score
Finally this is how we could get the Protein substitution score. We need to know that this UDF gets the MAX and the MIN values between all the posible scores for every variant. A possible execution could look like this:

In [6]:
df.select("id", protein_substitution("annotation", "polyphen").alias("polyphen"), protein_substitution("annotation", "sift")\
    .alias("sift")).where("polyphen[0]>=0").show()

+---------------+--------------+------------+
|             id|      polyphen|        sift|
+---------------+--------------+------------+
|22:25024098:G:A|[0.124, 0.581]| [0.0, 0.12]|
|22:28378810:T:G|[0.005, 0.005]|[0.37, 0.37]|
|22:29885644:C:A|[0.001, 0.001]|  [1.0, 1.0]|
|22:32831711:C:G|[0.027, 0.351]|[0.54, 0.78]|
+---------------+--------------+------------+

